In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import sys
import os
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Latex

# Add parent directory to path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
if current_dir not in sys.path:
    sys.path.append(current_dir)
    
from api_calls import get_ids_in_batch, get_processing_steps, get_efficiencies
from batch_selection import create_batch_selection
import process_handling
import access_token

url_base ="http://elnserver.lti.kit.edu"
url = f"{url_base}/nomad-oasis/api/v1"
token = access_token.get_token(url)
access_token.log_notebook_usage()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#initial widgets
batch_overview_output = widgets.HBox()
exclude_const = widgets.Checkbox(description = "hide nonvaried values")
exclude_string = widgets.Checkbox(description = "hide text values")
abbreviate_keys = widgets.Checkbox(description = "abbreviate value names")
detail_output = widgets.Output()
parameter_output = widgets.Output()

batch_data = [] 

def load_data_function(batch_ids_selector):
    global batch_data
    def create_batch_overview(batch_id):
        sample_ids = get_ids_in_batch(url, token, [batch_id])
        return process_handling.batch_process(get_processing_steps(url, token, sample_ids),
                                              get_efficiencies(url, token, sample_ids),
                                              batch_id,
                                              detail_output,
                                              parameter_output,
                                              exclude_const,
                                              exclude_string,
                                              abbreviate_keys)
    batch_overview_list = list(map(create_batch_overview, batch_ids_selector.value))

    batch_data = batch_overview_list
    batch_overview_output.children = batch_overview_list
    return

display(create_batch_selection(url, token, load_data_function))
display(batch_overview_output)
display(widgets.VBox([exclude_const,exclude_string,abbreviate_keys]))
display(detail_output)
display(parameter_output)

HBox()

Output()

Output()

In [3]:
# the global batch_data contains an object for every selected batch.
# The efficiencies attribute is a dict key value pairs sample_id:efficiency
# the get_selected_manufacturing_parameters retreives a dict containing dicts which store all the parameters for each sample
# example applications:
#pd.Series(batch_data[0].efficiencies)
#print(batch_data[0].get_selected_manufacturing_parameters(only_samples_with_efficiencies=False, only_samples_with_all_values=True))